In [1]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [ ]:
template = 'https://mx.indeed.com/jobs?q={}&l={}&advn=4703528095913257'

### Create a function that brings us the url

In [2]:
def get_url(position,location):
    template = 'https://mx.indeed.com/jobs?q={}&l={}'
    url = template.format(position,location)
    return url

In [16]:
url = get_url('programacion','Ciudad de México')

In [17]:
##Extract raw html

response = requests.get(url)
response

<Response [200]>

In [42]:
soup = BeautifulSoup(response.text,'html.parser')

In [19]:
cards = soup.find_all('div','jobsearch-SerpJobCard')

In [20]:
len(cards)

13

### Prototype the model with single record

In [21]:
card = cards[0]

In [22]:
atag = card.h2.a

In [23]:
job_title = atag.get('title')

In [31]:
job_url = 'https://mx.indeed.com' + atag.get('href')

In [25]:
company = card.find('span','company').text.strip()

In [28]:
job_location = card.find('div','recJobLoc').get('data-rc-loc')

In [35]:
job_summary = card.find('div','summary').text.strip().replace('\n',' ')

'Únete al mejor equipo de trabajo y vive la “Experiencia Play”. Tendrás oportunidad de desarrollar una carrera en el mejor casino de México.'

In [36]:
post_date = card.find('span','date').text

In [37]:
today = datetime.today().strftime('%Y-%m-%d')

'2021-01-24'

In [39]:
try:
    job_salaray = card.find('span','salaryText').text.strip()
except AttributeError:
    job_salary = ''

### Create master function

In [52]:
def get_record(card):
    atag = card.h2.a
    job_title = atag.get('title')
    job_url = 'https://mx.indeed.com' + atag.get('href')
    try:
        company = card.find('span','company').text.strip()
    except AttributeError:
        company = ''
    try:
        job_location = card.find('div','recJobLoc').get('data-rc-loc')
    except AttributeError:
        job_location = ''
    try:
        job_summary = card.find('div','summary').text.strip().replace('\n',' ')
    except AttributeError:
        job_summary = ''
    try:
        post_date = card.find('span','date').text
    except AttributeError:
        post_date = ''
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('span','salaryText').text.strip()
    except AttributeError:
        job_salary = ''
        
    record = (job_title,company,job_location,post_date,today,job_summary,job_salary,job_url)
    
    return record

In [53]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)

### How to obtain the next page?

In [59]:
while True:
    try:
        url = 'https://mx.indeed.com' + soup.find('a', {'aria-label': "Siguiente »"}).get('href')
    except AttributeError:
        break
        
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'jobsearch-SerpJobCard')
    
    for card in cards:
        record = get_record(card)
        records.append(record)

AttributeError: 'NoneType' object has no attribute 'text'

In [60]:
len(records)

376

In [61]:
with open('tech_jobs.csv', 'w', newline = '', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Title','Company','Location','Postdate','ExtractDate','Summary','Salary','JobURL'])
    writer.writerows(records)